In [1]:
import ee,geemap

Map = geemap.Map()

jx = geemap.shp_to_ee(r'C:\Users\xiong\Documents\geemap\shp\model\bound_dis.shp')
gx = geemap.shp_to_ee(r'C:\Users\xiong\Documents\geemap\shp\boundary\boundary2.shp')
label = 'Map'

# Map.addLayer(jx.geometry(),{},'jx')
# Map.addLayer(gx.geometry(),{},'gx')

In [2]:
lc = ee.ImageCollection("ESA/WorldCover/v100")\
                    .select(label)\
                    .mosaic()

# roi = ee.Geometry.Rectangle(122.347, 37.743, 122.024, 37.838)
sample = ee.FeatureCollection(lc.stratifiedSample(**{
                      'numPoints': 500,
                      'classBand': label,
                      'region': jx,
                      'scale': 500,
                      'geometries': True
                    })).filterMetadata(label,'equals',40)

# withRandom = sample.randomColumn()

# testingPartition = withRandom.filter(ee.Filter.gte('random', 0.8))

# Map.addLayer(sample,{},'sample')
# Map.addLayer(testingPartition,{},'testingPartition')
# Map

In [3]:
days=['31','28','31','30','31','30','31','31','30','31','30','31']
l8bands = ['B2','B3','B4','B5','B6','B7']
l7bands = ['B1','B2','B3','B4','B5','B7']
l5bands = ['B1','B2','B3','B4','B5','B6']
s2bands = ['B2','B3','B4','B8','B11','B12']
new_bands = ['B1','B2','B3','B4','B5','B6']
new_bands_ = ['B1','B2','B3','B4','B5','B6','NDVI','EVI','LSWI','NDWI','GCVI']
# new_bands_ = ['B1','B2','B3','B4','B5','B6','NDVI','EVI','GCVI']

crs = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA').first().select(0).projection().getInfo()['crs']
transform = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA').first().select(0).projection().getInfo()['transform']

# 去云函数
# Function to cloud mask from the pixel_qa band of Landsat 8 SR data.
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000)\
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"]) 

def maskL7sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('BQA')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to reflectance, without the QA bands.
    return image.updateMask(mask)\
      .select("B[0-8]*") \
      .copyProperties(image, ["system:time_start"]) 

def cloudfunction_L8(image):
    # image = ee.Algorithms.Landsat.simpleCloudScore(image)
    image = image.updateMask(image.select("cloud").lte(20))
    return image

def cloudfunction_ST2(image):
    # use add the cloud likelihood band to the image
    qa = image.select("QA60")
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000)\
        .select("B.*")\
        .copyProperties(image, ["system:index","system:time_start"])


    
def NDVI(img):
    ndvi = img.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return ndvi #return img.addBands(ndvi).updateMask(mask).clipToCollection(shp)


def EVI(img):
    nir = img.select("B4")
    red = img.select("B3")
    blue = img.select("B1")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return evi

def LSWI(img):
    lswi = img.normalizedDifference(['B4', 'B5']).rename('LSWI')
    return lswi

def NDWI(img):
    ndwi = img.normalizedDifference(['B2', 'B4']).rename('NDWI')
    return ndwi

def GCVI(img):
    nir = img.select("B4")
    green = img.select("B2")
    gcvi = img.expression("B4/B2-1",{"B4": nir,"B2": green})
    gcvi = gcvi.rename('GCVI')
    return gcvi

def RVI(img):
    nir = img.select("B4")
    red = img.select("B3")
    rvi = img.expression("B4/B3",{"B4": nir,"B3": red})
    rvi = rvi.rename('RVI')
    return rvi

def EVI2(img):
    nir = img.select("B4")
    red = img.select("B3")
    evi2 = img.expression("2.5*(B4 - B3)/(B4 + 2.4*B3 + 1)",{"B4": nir,"B3": red})
    evi2 = evi2.rename('EVI2')
    return evi2

def SAVI(img):
    nir = img.select("B4")
    red = img.select("B3")
    savi = img.expression("1.5*(B4 - B3)/(B4 + B3 + 0.5)",{"B4": nir,"B3": red})
    savi = savi.rename('SAVI')
    return savi

# def NDBI(img):
#    ndbi = img.normalizedDifference(['B6', 'B5']).rename('NDBI')
#    return img.addBands(ndbi)#img.addBands(ndbi).updateMask(mask).clipToCollection(shp)


# Unions Two Images
def unionimgs1(a,b,month):
    n = len(l8bands)
    for i in range(0,n,1):
        # print(l8bands[i])
        imgA = a.select(l8bands[i])
        imgB = b.select(s2bands[i])
        if i == 0:
            mask = imgA.unmask(imgB)
        else:
            mask = mask.addBands(imgA.unmask(imgB))
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).rename(new_bands)
    # return ee.Image(union.copyProperties(a, ["system:index","system:time_start"])).select(l8bands).rename(Nbands).unmask(-9999).clip(jx)

    # l7m
def unionimgs3(a,b,month):
    n = len(new_bands)
    for i in range(0,n,1):
        # print(l8bands[i])
        imgA = a.select(new_bands[i])
        imgB = b.select(l7bands[i])
        if i == 0:
            mask = imgA.unmask(imgB)
        else:
            mask = mask.addBands(imgA.unmask(imgB))
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).rename(new_bands)

# l5m
def unionimgs4(a,b,month):
    n = len(new_bands)
    for i in range(0,n,1):
        # print(l8bands[i])
        imgA = a.select(new_bands[i])
        imgB = b.select(l5bands[i])
        if i == 0:
            mask = imgA.unmask(imgB)
        else:
            mask = mask.addBands(imgA.unmask(imgB))
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).rename(new_bands)


    # Unions Two Images
def unionimgs2(a,b,month):
    abands = list(a.bandNames().getInfo())# abands = geemap.image_band_names(a)# abands = list(a.getInfo().get('bands'))是错的
    bbands = list(b.bandNames().getInfo())# bbands = geemap.image_band_names(b)# bbands = list(b.getInfo().get('bands'))是错的
    # print(abands)
    # print(bbands)
    if len(abands)==len(bbands):
        n = len(abands)
        # print(abands)
        # print(bbands)
        for i in range(0,n,1):
            # print(l8bands[i])
            imgA = a.select(abands[i])
            imgB = b.select(bbands[i])
            if i == 0:
                mask = imgA.unmask(imgB)
            else:
                mask = mask.addBands(imgA.unmask(imgB))
    else:
        print("ERROR  XXXXXXXXXXXXXXXXXXXXXXXXX")
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).select(abands).rename(new_bands)
    # return ee.Image(union.copyProperties(a, ["system:index","system:time_start"])).select(l8bands).rename(Nbands).unmask(-9999).clip(jx)

# 将image数据重新采样变为30m,使用原来数据的投影信息
def fun(image):
    crs = image.projection().getInfo()['crs']
    return image.resample('bilinear').reproject(**{'crs': crs,'scale': 30.0})

In [4]:
#参数初始化
Climit = 25 #云量
startY = 2017
endY   = 2017
startM=3
endM=11
visParams = {min: 0.2, max: 0.8, 'palette': ['blue', 'white','green']}

# 按年筛选
# imglist = [] #存放每月mosaic后的影像
img = ee.Image.constant(0).clip(jx)
b = []
for y in range(startY,startY+1,1):
    # 按月筛选
    for m in range(startM,endM+1,1):
        img = ee.Image.constant(0).clip(jx)
        print(m,end='-')
        ##===================================当年===================================================================
        #更换2月天数
        if (y//4==0):
            days[2]=29
        else:
            days[2]=28
        start_date = str(y) + '-' + str(m).zfill(2) + '-01'
        end_date = str(y) + '-' + str(m).zfill(2) + '-' + str(days[m-1])
        #l8
        l8m = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
                .sort('CLOUD_COVER', False)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(cloudfunction_L8)\
                .mosaic()#.clip(jx)#.filterMetadata('CLOUD_COVER', 'less_than', Climit)
        #S2
        s2m = ee.ImageCollection('COPERNICUS/S2')\
                        .filterDate(start_date, end_date)\
                        .filterBounds(jx)\
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', Climit))\
                        .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
                        .map(cloudfunction_ST2)\
                        .mosaic()#.clip(jx)
        
        l7m = ee.ImageCollection("LANDSAT/LE07/C01/T1_RT_TOA")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
                .sort('CLOUD_COVER', False)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(maskL7sr)\
                .mosaic()
        
#         l5m = ee.ImageCollection("LANDSAT/LT05/C01/T1_ANNUAL_TOA")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(ee.Algorithms.Landsat.simpleCloudScore)\
#                 .mosaic()
        
        union = unionimgs1(l8m,s2m,m)
        union = unionimgs3(union,l7m,m)
#         union = unionimgs4(union,l5m,m)
        
        
        ##===================================上一年===================================================================
        yy = y-1
        #更换2月天数
        if (yy//4==0):
            days[2]=29
        else:
            days[2]=28
        start_date_ = str(yy) + '-' + str(m-1).zfill(2) + '-16'
        end_date_ = str(yy) + '-' + str(m+1).zfill(2) + '-16'#+ str(days[m-1])
        
        #l8
        l8m_ = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
                .filterDate(start_date_, end_date_)\
                .filterBounds(jx)\
                .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
                .sort('CLOUD_COVER', False)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(cloudfunction_L8)\
                .mosaic()#.clip(jx)#.filterMetadata('CLOUD_COVER', 'less_than', Climit)
        
        #S2
        s2m_ = ee.ImageCollection('COPERNICUS/S2')\
                        .filterDate(start_date_, end_date_)\
                        .filterBounds(jx)\
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', Climit))\
                        .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
                        .map(cloudfunction_ST2)\
                        .mosaic()#clip(jx)#.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        
        l7m_ = ee.ImageCollection("LANDSAT/LE07/C01/T1_RT_TOA")\
                .filterDate(start_date_, end_date_)\
                .filterBounds(jx)\
                .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
                .sort('CLOUD_COVER', False)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(maskL7sr)\
                .mosaic()
        
#         l5m_ = ee.ImageCollection("LANDSAT/LT05/C01/T1_ANNUAL_TOA")\
#                 .filterDate(start_date_, end_date_)\
#                 .filterBounds(jx)\
#                 .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(ee.Algorithms.Landsat.simpleCloudScore)\
#                 .mosaic()
        
        union_= unionimgs1(l8m_,s2m_,m)
        union_ = unionimgs3(union_,l7m_,m)
#         union_ = unionimgs4(union_,l5m_,m)
        # print(list(union_.bandNames().getInfo()))
        # Map.addLayer(union_, {'bands': ['B4', 'B3', 'B2'], 'min': 0,  'max': 0.4}, 'S2_'+str(m))
        
        ##===================================下一年===================================================================
        yy = y+1
        #更换2月天数
        if (yy//4==0):
            days[2]=29
        else:
            days[2]=28
        start_date_ = str(yy) + '-' + str(m-1).zfill(2) + '-16'
        end_date_ = str(yy) + '-' + str(m+1).zfill(2) + '-'+str(days[m])
        
        #l8
        l8m__ = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
                .filterDate(start_date_, end_date_)\
                .filterBounds(jx)\
                .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
                .sort('CLOUD_COVER', False)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(cloudfunction_L8)\
                .mosaic()#.clip(jx)#.filterMetadata('CLOUD_COVER', 'less_than', Climit)
        
        #S2
        s2m__ = ee.ImageCollection('COPERNICUS/S2')\
                        .filterDate(start_date_, end_date_)\
                        .filterBounds(jx)\
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', Climit))\
                        .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
                        .map(cloudfunction_ST2)\
                        .mosaic()#.clip(jx)#.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        
        l7m__ = ee.ImageCollection("LANDSAT/LE07/C01/T1_RT_TOA")\
                .filterDate(start_date_, end_date_)\
                .filterBounds(jx)\
                .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
                .sort('CLOUD_COVER', False)\
                .map(ee.Algorithms.Landsat.simpleCloudScore)\
                .map(maskL7sr)\
                .mosaic()
        
#         l5m__ = ee.ImageCollection("LANDSAT/LT05/C01/T1_ANNUAL_TOA")\
#                 .filterDate(start_date_, end_date_)\
#                 .filterBounds(jx)\
#                 .filterMetadata('CLOUD_COVER', 'less_than', Climit)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(ee.Algorithms.Landsat.simpleCloudScore)\
#                 .mosaic()
        
        union__ = unionimgs1(l8m__,s2m__,m)
        union__ = unionimgs3(union__,l7m__,m)
#         union__ = unionimgs4(union__,l5m__,m)
        # Map.addLayer(union_, {'bands': ['B4', 'B3', 'B2'], 'min': 0,  'max': 0.4}, 'S2_'+str(m))
        
        ##===================================合并==========================================================================
        Union = unionimgs2(union,union_,m)
        Union = unionimgs2(Union,union__,m)#.unmask(-9999)#.clip(jx)
        
        # Vegetaion Index
        Union = Union.addBands( NDVI(Union))
        Union = Union.addBands( EVI(Union) )
        Union = Union.addBands( LSWI(Union))
        Union = Union.addBands( NDWI(Union))
        Union = Union.addBands( GCVI(Union))
#         Union = Union.addBands( RVI(Union) )
#         Union = Union.addBands( EVI2(Union))
#         Union = Union.addBands( SAVI(Union))
        # print(Union.bandNames().getInfo()) 
        
#         Map.addLayer(Union, {'bands': ['B3', 'B2', 'B1'], 'min': 0,  'max': 0.4}, 'U_'+str(m)) #.randomVisualizer()
        
        
        
        # ==========================Collecting monthly imageBands to 'img'==============================
        import os
        import pandas as pd
        img = img.addBands(Union)
        image = img.select(['NDVI','LSWI'])
        outfile = r'C:\Users\xiong\Downloads\ndvi1'+os.sep+str(m)+'.csv'
        
        if os.path.exists(outfile):
            print('file already exits')
            continue
        
        # 按点提取值
        r_shp = image.sampleRegions(collection=sample, scale=500, geometries=True)
        
#         temp = image.select('NDVI').sampleRegions(collection=sample, scale=500, geometries=True)
        
        break
        arr = r_shp.reduceColumns(ee.Reducer.toList(2), ['NDVI','LSWI']).values().get(0).getInfo()
        df = pd.DataFrame(arr,columns=['NDVI','LSWI'])
        
        df.to_csv(outfile,index=False,header=True)
        
#         # dem
#         dem = ee.Image('USGS/SRTMGL1_003').clip(jx)
#         img = img.addBands(dem)
        
#         #slope
#         slope = ee.Terrain.slope(dem).lt(10)
#         img = img.addBands(slope)
        
    # ===================Rename all bands name=======================
#     newB = ['NONE']
#     for x in range(startM,endM+1):
#         for y in new_bands_:
#             newB.append(str(x)+'M_'+y)
#     img = img.rename(newB)
#     # print(len(img.getInfo().get('bands')))
#     # print(newB)

# # # Map

3-

In [7]:
# Map = geemap.Map()
# Map.addLayer(sample,{},'sample')
# Map.addLayer(image.randomVisualizer(),{},'img')
# Map

In [8]:
# r_shp.getInfo()